In [1]:
import fitz  # PyMuPDF
import os

In [4]:
input_pdf = "/Users/angusmclean/Downloads/Philly Savory Insights for WPP copy/Situation Analysis and Equity/Philadelphia in Plain Herbs Flavored 2024 final shared.pptx"  # Replace with your actual PDF file path

In [5]:
import fitz  # PyMuPDF
import os

def extract_mac_preview_highlights(pdf_path):
    """
    Extracts highlights and annotations from a PDF file, including those made in macOS Preview.
    
    :param pdf_path: Path to the input PDF file.
    """
    try:
        doc = fitz.open(pdf_path)
        annotations = []
        output_txt_path = os.path.splitext(pdf_path)[0] + ".txt"

        for page_num, page in enumerate(doc, start=1):
            words = page.get_text("words")  # Extract words from the page
            words_dict = {fitz.Rect(w[:4]): w[4] for w in words}  # Create a dictionary of words
            
            for annot in page.annots():  # Loop through annotations
                highlight_text = ""
                annot_text = annot.info.get("content", "").strip()  # User-added note (if any)

                if annot.type[0] == 8:  # 8 = Highlight annotation
                    quad_points = annot.vertices  # Get the highlight region
                    if quad_points:
                        for i in range(0, len(quad_points), 4):
                            rect = fitz.Quad(quad_points[i:i+4]).rect  # Convert highlight coordinates to a rectangle
                            highlight_text += " ".join(
                                words_dict[w] for w in words_dict if w.intersects(rect)
                            ) + " "  # Extract words inside the highlight

                # Save annotation details
                if highlight_text.strip() or annot_text:
                    annotations.append(
                        f"Page {page_num}:\nHighlighted Text: {highlight_text.strip()}\nAnnotation: {annot_text}\n---\n"
                    )

        # Save extracted highlights to a text file
        if annotations:
            with open(output_txt_path, "w", encoding="utf-8") as f:
                f.writelines(annotations)
            print(f"Annotations successfully extracted to {output_txt_path}")
        else:
            print("No highlights or annotations found.")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    extract_mac_preview_highlights(input_pdf)


No highlights or annotations found.
